작물 잎 사진으로 질병 분류하기 - 출처:

In [2]:
import os
import shutil

dataset_dir = "D:/dataset"
classes_list = os.listdir(dataset_dir)

base_dir = "D:/splitted"
os.makedirs(base_dir,exist_ok=True)

train_dir = os.path.join(base_dir,"train")
os.makedirs(train_dir,exist_ok=True)
validation_dir = os.path.join(base_dir, "validation")
os.makedirs(validation_dir,exist_ok=True)
test_dir = os.path.join(base_dir, "test")
os.makedirs(test_dir,exist_ok=True)

classes_list

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Cherry___healthy',
 'Cherry___Powdery_mildew',
 'Corn___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn___Common_rust',
 'Corn___healthy',
 'Corn___Northern_Leaf_Blight',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___healthy',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___healthy',
 'Potato___Late_blight',
 'Strawberry___healthy',
 'Strawberry___Leaf_scorch',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___healthy',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [3]:
for cls in classes_list:
    os.makedirs(os.path.join(train_dir,cls),exist_ok=True)
    os.makedirs(os.path.join(validation_dir,cls),exist_ok=True)
    os.makedirs(os.path.join(test_dir,cls),exist_ok=True)
    

In [4]:
import math
import shutil

for cls in classes_list:
    path = os.path.join(dataset_dir,cls)
    fnames = os.listdir(path)

    train_size = math.floor(len(fnames)*0.6)
    validation_size = math.floor(len(fnames)*0.2)
    test_size = math.floor(len(fnames)*0.2)


    train_fnames = fnames[:train_size]
    print(f"Train size({cls}): {len(train_fnames)}")
    for fname in fnames:
        src = os.path.join(path,fname)
        dst = os.path.join(train_dir,cls,fname)
        shutil.copyfile(src,dst)
    
    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print(f"validation size({cls}): {len(validation_fnames)}")
    for fname in fnames:
        src = os.path.join(path,fname)
        dst = os.path.join(validation_dir,cls,fname)
        shutil.copyfile(src,dst)

    test_fnames = fnames[(validation_size + train_size):]
    print(f"test size({cls}): {len(test_fnames)}")
    for fname in fnames:
        src = os.path.join(path,fname)
        dst = os.path.join(test_dir,cls,fname)
        shutil.copyfile(src,dst)

Train size(Apple___Apple_scab): 378
validation size(Apple___Apple_scab): 126
test size(Apple___Apple_scab): 126
Train size(Apple___Black_rot): 372
validation size(Apple___Black_rot): 124
test size(Apple___Black_rot): 125
Train size(Apple___Cedar_apple_rust): 165
validation size(Apple___Cedar_apple_rust): 55
test size(Apple___Cedar_apple_rust): 55
Train size(Apple___healthy): 987
validation size(Apple___healthy): 329
test size(Apple___healthy): 329
Train size(Cherry___healthy): 512
validation size(Cherry___healthy): 170
test size(Cherry___healthy): 172
Train size(Cherry___Powdery_mildew): 631
validation size(Cherry___Powdery_mildew): 210
test size(Cherry___Powdery_mildew): 211
Train size(Corn___Cercospora_leaf_spot Gray_leaf_spot): 307
validation size(Corn___Cercospora_leaf_spot Gray_leaf_spot): 102
test size(Corn___Cercospora_leaf_spot Gray_leaf_spot): 104
Train size(Corn___Common_rust): 715
validation size(Corn___Common_rust): 238
test size(Corn___Common_rust): 239
Train size(Corn___h

In [8]:
!pip install torchvision

     ---------------------------------------- 1.2/1.2 MB 4.7 MB/s eta 0:00:00
     -------------------------------------- 172.3/172.3 MB 5.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1


In [3]:
import torch

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

BATCH_SIZE = 256
EPOCHS = 30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

transform_base = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])

train_dataset = ImageFolder(train_dir, transform=transform_base)
validation_dataset = ImageFolder(validation_dir, transform=transform_base)

from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
vallidation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):

    def __init__(self):

        super(Net,self).__init__()

        self.conv1 = nn.Conv2d(3,32,3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2  = nn.Conv2d(32,64,3, padding=1)
        self.conv3 = nn.Conv2d(64,64,3, padding=1)

        self.fc1 = nn.Linear(64*8*8, 512)
        self.fc2 = nn.Linear(512, 33)

    def forward(self,x):

        x = self.pool(F.relu(self.conv1(x)))
        x= F.dropout(x, p=0.25, training=self.training)

        x = self.pool(F.relu(self.conv2(x)))
        x = F.dropout(x, p=0.25, training=self.training)

        x = self.pool(F.relu(self.conv3(x)))
        x = F.dropout(x, p=0.25, training=self.training)

        x = x.view(-1,64*8*8)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)
    
model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

In [5]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data,target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()       
        output = model(data)
        loss = F.cross_entropy(output, target)      #classification = cross_entropy
        loss.backward()
        optimizer.step()

In [6]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0 

    with torch.no_grad():   
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output,target,reduction='sum').item()
            pred = output.max(1,keepdim=True)[1]    
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [7]:
import time
import copy

def train_model(model, train_loader, test_loader, optimizer, num_epochs=30):
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(1, num_epochs + 1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, test_loader)

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
    
        time_elapsed = time.time() - since
        print('Epoch: {}  Train Loss: {:.4f}  Train Acc: {:.2f}%  Val Loss: {:.4f}  Val Acc: {:.2f}%  Time: {:.0f}m {:.0f}s'.format(epoch, train_loss, train_acc, val_loss, val_acc, time_elapsed // 60, time_elapsed % 60))

    model.load_state_dict(best_model_wts)
    return model

model_base = train_model(model_base, train_loader, vallidation_loader, optimizer, EPOCHS)
torch.save(model_base, 'beseline.pt')




Epoch: 1  Train Loss: 1.3815  Train Acc: 57.86%  Val Loss: 1.3815  Val Acc: 57.86%  Time: 4m 14s
Epoch: 2  Train Loss: 0.7826  Train Acc: 76.60%  Val Loss: 0.7826  Val Acc: 76.60%  Time: 4m 4s
Epoch: 3  Train Loss: 0.5393  Train Acc: 83.58%  Val Loss: 0.5393  Val Acc: 83.58%  Time: 3m 55s
Epoch: 4  Train Loss: 0.4539  Train Acc: 85.44%  Val Loss: 0.4539  Val Acc: 85.44%  Time: 4m 1s
Epoch: 5  Train Loss: 0.3858  Train Acc: 88.08%  Val Loss: 0.3858  Val Acc: 88.08%  Time: 4m 8s
Epoch: 6  Train Loss: 0.3190  Train Acc: 90.05%  Val Loss: 0.3190  Val Acc: 90.05%  Time: 3m 40s
Epoch: 7  Train Loss: 0.2680  Train Acc: 91.92%  Val Loss: 0.2680  Val Acc: 91.92%  Time: 3m 52s
Epoch: 8  Train Loss: 0.2812  Train Acc: 90.90%  Val Loss: 0.2812  Val Acc: 90.90%  Time: 3m 46s
Epoch: 9  Train Loss: 0.2533  Train Acc: 92.19%  Val Loss: 0.2533  Val Acc: 92.19%  Time: 3m 50s
Epoch: 10  Train Loss: 0.2139  Train Acc: 93.30%  Val Loss: 0.2139  Val Acc: 93.30%  Time: 4m 0s
Epoch: 11  Train Loss: 0.1845  Tr